**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

import tensorflow as tf
from keras import backend as K

config = tf.ConfigProto(intra_op_parallelism_threads=5,\
        inter_op_parallelism_threads=5, allow_soft_placement=True,\
        device_count = {'CPU' : 1, 'GPU' : 1})
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e, decay=0):
        self.epsilon = e * (1 - decay)

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The function act takes as an input a state `s`, and it returns the action to perform. In order to ensure a good balance between exploration and exploitation, the epsilon greedy strategy is used. This strategy chooses a random action with probabily $\epsilon$, and the optimal learned action with probability $1 - \epsilon$. This allows the algorithm to explore all the actions in all the states and thus will know which action is the best to choose in which state.

> $\epsilon$ is essential, because without it, the agent might stuck in a suboptimal but local solution depending on his previous seen states.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 # typo

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #typo
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 50 # set small when debugging
epochs_test= 20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> Position is an array indicating the explorabale environnement of the rat, it has 0 where the rat can go, -1 where it can't go, and 1 indicating the position of the rat.

> Board is an array indicating the reward (bonus or malus) that the rat will get if he goes to a certain position. Bonuses and maluses are drown from a binomial distribution with a probability = temperature.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0] 

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward
        
        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [16]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.0/15.0. Average score (-7.0)
Win/lose count 12.0/16.0. Average score (-5.5)
Final score: -5.5


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer__


 __1)__ Let $\pi$ a policy, we have:

\begin{align*}
Q^{\pi}(s,a)& =E[ \sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& = E[ r_t(s,a) + \sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& = r_t(s,a) + \gamma E[ \sum_{k=0}^{\infty} \gamma^k r_{t+k+1}|s_{t} = s, a_{t} = a] \\
& =  r_t(s,a) + \gamma E[E[\sum_{k=0}^{\infty}[\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']|s_t = s, a_t = a] \text{ by conditioning on }(s_{t+1}=s',a_{t+1}=a')\\
& =  r_t(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )[E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
& =  \sum_{(s',a')} (p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) r_t(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}

__2)__ We have

\begin{align*}
\max_\pi Q^{\pi}(s,a)&=\max_\pi E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\\
&=\max_\pi \sum_{s', a'} \pi(a'|s) p(s'|s,a)[r(s,a)+\gamma Q^{\pi}(s',a')]\\
&=\sum_{s', a'} \pi^*(a'|s) p(s'|s,a)[r(s,a)+\gamma Q^{*}(s',a')] \text{ ( $\pi^*$ is stable by bellman operator )}\\
&=\sum_{s'} p(s'|s,a)[r(s,a)+\gamma \max_a' Q^{*}(s',a')]\\
&=E_{s'\sim p(s'|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{align*}

__3)__ We want to approximate $Q^{\pi}$ using a function approximator $Q(s, a; \theta) \approx Q^{*}(s,a)$, where $\theta$ is a parameter . A typical function approximator is neural networks, and in order to determine the parameters of the network, we need a loss function. The latter can be written in the form: $L_i(\theta_i) = E_{s' \sim \pi^*(.|s,a)}[(y_i - Q(s, a; \theta_i))^2]$. The target value $y_i$ must be choosen in a way that mimic the behavior of the standard Q-learning update rule, i.e. $y_i = r(s,a)+\gamma \max_{a'} Q(s',a'; \theta_{i-1})$. By doing so, we ensure that, at each iteration, our  $y_i$  roughly converges towards the optimal Q-value function.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
from collections import deque

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        # deque is a list FIFO with a fixed max length
        self.memory = deque(maxlen=max_memory)

    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory), size=1)[0]]

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [72]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            input_states[i] = s_
            target_q[i] =self.model.predict(s_.reshape([1, s_.shape[0], s_.shape[1], s_.shape[2]]))[0]
            
            # setting the target values depending on if the state is terminal or not(cf DQN Algorithm)
            if game_over_:
                target_q[i,a_] = r_
            else:
                target_q[i,a_] = r_  + self.discount * max(self.model.predict(n_s_.reshape([1, s_.shape[0], s_.shape[1], s_.shape[2]]))[0])

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(20, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Flatten())
        model.add(Dense(30, activation ='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [68]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0276 | Win/lose count 10.5/15.0 (-4.5)
Epoch 001/050 | Loss 0.0115 | Win/lose count 4.0/4.0 (0.0)
Epoch 002/050 | Loss 0.0051 | Win/lose count 10.0/5.0 (5.0)
Epoch 003/050 | Loss 0.0021 | Win/lose count 2.5/1.0 (1.5)
Epoch 004/050 | Loss 0.0033 | Win/lose count 5.0/3.0 (2.0)
Epoch 005/050 | Loss 0.0020 | Win/lose count 7.5/2.0 (5.5)
Epoch 006/050 | Loss 0.0578 | Win/lose count 8.5/4.0 (4.5)
Epoch 007/050 | Loss 0.0054 | Win/lose count 11.0/2.0 (9.0)
Epoch 008/050 | Loss 0.0054 | Win/lose count 15.0/6.0 (9.0)
Epoch 009/050 | Loss 0.0027 | Win/lose count 13.5/5.0 (8.5)
Epoch 010/050 | Loss 0.0037 | Win/lose count 5.5/3.0 (2.5)
Epoch 011/050 | Loss 0.0035 | Win/lose count 5.0/2.0 (3.0)
Epoch 012/050 | Loss 0.0507 | Win/lose count 12.0/3.0 (9.0)
Epoch 013/050 | Loss 0.0020 | Win/lose count 8.0/2.0 (6.0)
Epoch 014/050 | Loss 0.0013 | Win/lose count 15.0/4.0 (11.0)
Epoch 015/050 | Loss 0.0046 | Win/lose count 10.5/0 (10.5)
Epoch 016/050 | Loss 0.0030 | Win/lose count 9

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [75]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # Inspired from DQN architecture
        model = Sequential()
        model.add(Conv2D(16, (3, 3), input_shape=(5,5,self.n_state,), activation='relu'))
        model.add(Conv2D(32, (3, 3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dense(4))
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile("adam", "mse")
        self.model = model

In [48]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,50,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0089 | Win/lose count 5.0/5.0 (0.0)
Epoch 001/050 | Loss 0.0586 | Win/lose count 1.5/2.0 (-0.5)
Epoch 002/050 | Loss 0.0005 | Win/lose count 1.0/2.0 (-1.0)
Epoch 003/050 | Loss 0.0017 | Win/lose count 2.5/3.0 (-0.5)
Epoch 004/050 | Loss 0.0012 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/050 | Loss 0.0019 | Win/lose count 4.5/1.0 (3.5)
Epoch 006/050 | Loss 0.0034 | Win/lose count 5.5/4.0 (1.5)
Epoch 007/050 | Loss 0.0023 | Win/lose count 5.5/6.0 (-0.5)
Epoch 008/050 | Loss 0.0367 | Win/lose count 3.0/5.0 (-2.0)
Epoch 009/050 | Loss 0.0043 | Win/lose count 1.0/1.0 (0.0)
Epoch 010/050 | Loss 0.0023 | Win/lose count 3.0/1.0 (2.0)
Epoch 011/050 | Loss 0.0422 | Win/lose count 5.5/0 (5.5)
Epoch 012/050 | Loss 0.0009 | Win/lose count 2.0/0 (2.0)
Epoch 013/050 | Loss 0.0023 | Win/lose count 5.5/2.0 (3.5)
Epoch 014/050 | Loss 0.0016 | Win/lose count 8.0/1.0 (7.0)
Epoch 015/050 | Loss 0.0015 | Win/lose count 8.5/1.0 (7.5)
Epoch 016/050 | Loss 0.0014 | Win/lose count 5.0/4.0 (1

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [76]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 7.5/3.0. Average score (4.5)
Win/lose count 13.5/1.0. Average score (8.5)
Win/lose count 2.0/2.0. Average score (5.666666666666667)
Win/lose count 13.0/2.0. Average score (7.0)
Win/lose count 21.0/5.0. Average score (8.8)
Win/lose count 3.0/1.0. Average score (7.666666666666667)
Win/lose count 16.5/3.0. Average score (8.5)
Win/lose count 14.0/3.0. Average score (8.8125)
Win/lose count 15.5/3.0. Average score (9.222222222222221)
Win/lose count 10.0/2.0. Average score (9.1)
Win/lose count 12.5/1.0. Average score (9.318181818181818)
Win/lose count 17.0/1.0. Average score (9.875)
Win/lose count 9.5/2.0. Average score (9.692307692307692)
Win/lose count 5.5/4.0. Average score (9.107142857142858)
Win/lose count 10.0/3.0. Average score (8.966666666666667)
Win/lose count 6.5/0. Average score (8.8125)
Win/lose count 14.0/4.0. Average score (8.882352941176471)
Win/lose count 21.0/4.0. Average score (9.333333333333334)
Win/lose count 19.0/3.0. Average score (9.684210

In [30]:
HTML(display_videos('cnn_test10.mp4'))

In [31]:
HTML(display_videos('fc_test10.mp4'))

> It can be seen that the CNN architecture gives better results than the FC architecture, even if we used the same hyperparameters, and this is valid for all temperatures. However, we noted that  both algorithms tend to not explore all the maps, and get stucked in the same region because they don't have visibility on all the maps (This behaviour is very noticeable at low temperatures). Also we noticed that the agent keep exploring areas that he had explored before. This issues can be resolved by, first reducing the exploration rate over episodes, and second by adding a malus if the agent visit a cell which is already visited, so this will prevent the behaviour of getting stucked in the same region.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [12]:
def train_explore(agent, env, epoch, decay=0.25, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        epsilon = agent.epsilon
        agent.set_epsilon(epsilon, decay)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
            
        self.malus_position[self.x, self.y] += 0.1
        reward = reward + self.board[self.x, self.y]   
        
        self.board[self.x, self.y] = 0 # erase reward from board
        game_over = self.t > self.max_time # game over if more than 500 steps

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        # set bonus
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        # set malus
        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        
        # lines added (put 0 poison/cheese outside the border)
        self.board[:2,:] = 0
        self.board[-2:,:] = 0
        self.board[:, :2] = 0
        self.board[:, -2:] = 0

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 # error, 2 times same line
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [63]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 50, decay=0.05, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0321 | Win/lose count 10.5/57.30000000000004 (-46.80000000000004)
Epoch 001/050 | Loss 0.0081 | Win/lose count 17.0/33.30000000000002 (-16.30000000000002)
Epoch 002/050 | Loss 0.0106 | Win/lose count 20.5/23.90000000000001 (-3.4000000000000092)
Epoch 003/050 | Loss 0.0263 | Win/lose count 17.0/32.40000000000004 (-15.400000000000041)
Epoch 004/050 | Loss 0.0084 | Win/lose count 18.0/28.50000000000004 (-10.500000000000039)
Epoch 005/050 | Loss 0.0479 | Win/lose count 18.0/23.700000000000006 (-5.700000000000006)
Epoch 006/050 | Loss 0.0149 | Win/lose count 23.0/24.8 (-1.8000000000000007)
Epoch 007/050 | Loss 0.0145 | Win/lose count 18.5/18.4 (0.10000000000000142)
Epoch 008/050 | Loss 0.0102 | Win/lose count 21.0/20.59999999999998 (0.4000000000000199)
Epoch 009/050 | Loss 0.0086 | Win/lose count 25.0/17.49999999999999 (7.500000000000011)
Epoch 010/050 | Loss 0.0146 | Win/lose count 23.0/19.500000000000004 (3.4999999999999964)
Epoch 011/050 | Loss 0.0630 | Win/lose co

In [65]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 26.5/3.0. Average score (23.5)
Win/lose count 25.5/6.0. Average score (21.5)
Win/lose count 24.0/3.0. Average score (21.333333333333332)
Win/lose count 30.5/3.0. Average score (22.875)
Win/lose count 22.5/6.0. Average score (21.6)
Win/lose count 18.0/3.0. Average score (20.5)
Win/lose count 23.5/3.0. Average score (20.5)
Win/lose count 27.0/2.0. Average score (21.0625)
Win/lose count 27.5/3.0. Average score (21.444444444444443)
Win/lose count 22.0/9.0. Average score (20.6)
Win/lose count 27.5/1.0. Average score (21.136363636363637)
Win/lose count 25.0/4.0. Average score (21.125)
Win/lose count 25.0/3.0. Average score (21.192307692307693)
Win/lose count 17.5/3.0. Average score (20.714285714285715)
Win/lose count 20.0/6.0. Average score (20.266666666666666)
Win/lose count 18.0/11.0. Average score (19.4375)
Win/lose count 23.5/7.0. Average score (19.264705882352942)
Win/lose count 25.5/7.0. Average score (19.22222222222222)
Win/lose count 25.0/2.0. Average score (19.4210526

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***